In [34]:
import folium
import json

# Carga el archivo .geojson
with open('STC_Metro_estaciones.geojson', encoding='utf-8') as f:
    data = json.load(f)

# Centrar el mapa en la ciudad de México
mapa = folium.Map(location=[19.4163340855965, -99.0747357270116], zoom_start=12, tiles='CartoDB positron')

# Crear un diccionario para almacenar las estaciones agrupadas por línea
lineas = {}

lineasColores = {
    "01": "#E30A45",  # Rosa mexicano
    "02": "#0000FF",  # Azul
    "03": "#808000",  # Verde olivo
    "04": "#00FFFF",  # Cian
    "05": "#FFFF00",  # Amarillo    
    "06": "#FF0000",  # Rojo 
    "07": "#FFA500",  # Naranja
    "08": "#008000",  # Verde
    "09": "#A52A2A",  # Cafe
    "A": "#800080",   # Morado
    "B": "#696969",   # Gris
    "12": "#FFD700",  # Oro
}

# Agrupar estaciones por línea
for feature in data['features']:
    nombre = feature['properties']['NOMBRE']
    linea = feature['properties']['LINEA']
    latitud, longitud = feature['geometry']['coordinates'][::-1]
    if linea not in lineas:
        lineas[linea] = []
    lineas[linea].append((latitud, longitud))
    
    folium.CircleMarker(
        location=(latitud, longitud),
        radius=5,
        color=lineasColores.get(linea, '#000000'),
        fill=True,
        fill_color=lineasColores.get(linea, '#000000'),
        fill_opacity=0.7,
        popup=nombre
    ).add_to(mapa)

# Agregar polilíneas para cada línea
for linea, estaciones in lineas.items():
    color = lineasColores.get(linea, '#000000') 
    folium.PolyLine(estaciones, color=color, weight=2.5, opacity=1, popup=linea).add_to(mapa)

# Mostrar el mapa
mapa
